In [1]:
import random
import pickle
from datetime import datetime
from random import randint
import pandas as pd

In [2]:
def loadfile(filename):
    pickle_in = open("C:/Users/Akarsh/Downloads/DP_scripts/store_emb/" + filename + ".pickle","rb")
    return pickle.load(pickle_in)

def save2file(filename, data):
    pickle_out = open("C:/Users/Akarsh/Downloads/DP_scripts/store_emb/" + filename + ".pickle", "wb")
    pickle.dump(data, pickle_out, protocol=pickle.HIGHEST_PROTOCOL)
    pickle_out.close()

main_str = str(437)
filename = "ip2vec_emb_" + main_str
ip2vec_emb = loadfile(filename)

filename = "updated_train_" + main_str
updated_train = loadfile(filename)

display(updated_train.head())
print(updated_train.shape)

,Day,Time,Duration,Proto,Src IP Addr,Src Pt,Dst IP Addr,Dst Pt,Packets,Bytes,Flags,class,attackType
0,0,11:03:29.789,0.00,UDP,192.168.220.10,42968,DNS,53,2,176,......,normal,---
1,0,16:54:11.971,0.64,TCP,10086_192,80,192.168.220.14,43392,6,3230,.AP...,normal,---
2,4,18:19:42.203,0.00,TCP,16569_82,80,192.168.220.8,41440,1,66,.A....,normal,---
3,5,11:08:00.553,0.00,TCP,10296_238,443,192.168.200.9,59692,2,154,.AP...,normal,---
4,5,17:51:09.047,0.00,TCP,192.168.220.14,52236,10056_243,443,1,66,.A....,normal,---


(260391, 13)


In [3]:
ipToEmbed = dict()
for row in ip2vec_emb.values:
    key = row[20].strip()
    ipToEmbed[key] = list(row[:20])
print("created emb dict")

del ip2vec_emb

gan_train = []; counter = 0;
for row in updated_train.values:
    counter += 1
    if counter % 50000 == 0:
        print("Read lines :", counter)
        
    day = str(row[0]).strip()
    time_ = row[1].strip()
    dur = str(row[2]).strip() + "_d"
    proto = row[3].strip()
    srcIP = row[4].strip()
    srcPt = str(row[5]).strip() + "_p"
    dstIP = row[6].strip()
    dstPt = str(row[7]).strip() + "_p"
    packets = str(row[8]).strip() + "_k"
    bytzes = str(row[9]).strip() + "_b"
    flags = row[10].strip()
    clazz = row[11].strip()
    
    rl = []
    # date and time
    for i in range(0,7):
        rl.append(0)
    rl[int(day)] = 1
    datetime_object = datetime.strptime(time_, "%H:%M:%S.%f") # add time # Handle timestamp
    seconds = float(datetime_object.hour * 3600.0 + datetime_object.minute * 60.0 + datetime_object.second) / 86400.0
    rl.append(seconds)
    
    # duration
    du = ipToEmbed[dur]
    for a in du:
        rl.append(float(a))

    # transport protocol
    icmp = 0; udp = 0; tcp = 0
    if proto == "ICMP":
        icmp = icmp + 1
    if proto == "TCP":
        tcp = tcp + 1
    if proto == "UDP":
        udp = udp + 1
    rl.append(icmp); rl.append(udp); rl.append(tcp)

    # Replace IPs and Ports with their embedding
    sIP = ipToEmbed[srcIP]; sp = ipToEmbed[srcPt]; dIP = ipToEmbed[dstIP]; dp = ipToEmbed[dstPt]
    for a in sIP:
        rl.append(float(a))
    for a in sp:
        rl.append(float(a))
    for a in dIP:
        rl.append(float(a))
    for a in dp:
        rl.append(float(a))

    # Packet # Bytes 
    pk = ipToEmbed[packets]; by = ipToEmbed[bytzes]
    for a in pk:
        rl.append(float(a))
    for a in by:
        rl.append(float(a))

    # binarize flags
    syn = 0; ack = 0; fin = 0; urg = 0; psh = 0; res = 0
    if "S" in flags:
        syn = syn + 1
    if "A" in flags:
        ack = ack + 1
    if "F" in flags:
        fin = fin + 1
    if "U" in flags:
        urg = urg + 1
    if "P" in flags:
        psh = psh + 1
    if "R" in flags:
        res = res + 1
    rl.extend([syn,ack,fin,urg,psh,res])
    
     # clazz 
    normal = 0; attacker = 0; victim = 0
    if clazz == "normal": 
        normal = normal + 1
    if clazz == "attacker":
        attacker = attacker + 1
    if clazz == "victim":
        victim = victim + 1
    rl.extend([normal,attacker,victim])
    gan_train.append(rl)

created emb dict


In [4]:
gan_data = pd.DataFrame(gan_train)
display(gan_data.head())

filename = "gan_train_" + main_str
save2file(filename, gan_data)

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
0,1,0,0,0,0,0,0,0.460752,0.690727,0.289209,...,0.459836,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0.704294,0.457027,0.217206,...,0.565490,0,1,0,0,1,0,1,0,0
2,0,0,0,0,1,0,0,0.763681,0.690727,0.289209,...,0.314966,0,1,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0.463889,0.690727,0.289209,...,0.681866,0,1,0,0,1,0,1,0,0
4,0,0,0,0,0,1,0,0.743854,0.690727,0.289209,...,0.314966,0,1,0,0,0,0,1,0,0
